<a href="https://colab.research.google.com/github/jrleonett/Analisis_de_Voz_con_Praat/blob/main/compracion_audios_praat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis Forense de Voz con PRAAT

**Herramienta de Comparación de Voces con Base Científica**

Este programa está diseñado para realizar análisis forenses de voz utilizando técnicas avanzadas de fonética con la librería PRAAT. Permite comparar dos muestras de audio, detectar emociones, estimar edad y sexo, y determinar si las voces pertenecen a la misma persona. Además, genera gráficos detallados para facilitar la interpretación de los resultados.

### Características Principales:
- **Conversión Automática a WAV**: Convierte archivos de audio en formatos comprimidos (MP3, M4A, etc.) a WAV para un análisis preciso.
- **Detección de Emociones**: Identifica emociones como felicidad, tristeza, enojo o neutralidad basándose en el tono (pitch) y la intensidad.
- **Comparación de Formantes y MFCC**: Utiliza técnicas robustas como MFCC (Mel-Frequency Cepstral Coefficients) y DTW (Dynamic Time Warping) para comparar patrones de voz.
- **Gráficos Detallados**: Incluye oscilogramas, gráficos de pitch, intensidad, formantes y espectrogramas para un análisis visual completo.
- **Información de Archivos**: Muestra detalles de los archivos originales y convertidos para mantener la transparencia y la integridad forense.

### Desarrollado por:
José R. Leonett para el **Grupo de Peritos Forenses Digitales de Guatemala**  
🌐 [www.forensedigital.gt](http://www.forensedigital.gt)

---

**Instrucciones de Uso:**
1. Sube dos archivos de audio en los campos correspondientes.
2. Haz clic en "Submit" para iniciar el análisis.
3. Revisa los resultados y gráficos generados.

In [12]:
#@title Análisis Forense de Voz con PRAAT
# %%capture
!nvidia-smi

# Instalar dependencias necesarias
!pip install gradio numpy matplotlib praat-parselmouth pydub librosa fastdtw

# Importar bibliotecas
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
import parselmouth  # Para interactuar con PRAAT
import os
from datetime import datetime
from pydub import AudioSegment
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import librosa

# Función para convertir cualquier archivo de audio a WAV
def convert_to_wav(filepath):
    audio = AudioSegment.from_file(filepath)
    wav_path = filepath.rsplit('.', 1)[0] + "_converted.wav"
    audio.export(wav_path, format="wav")
    return wav_path

# Función para detectar emociones a partir de pitch e intensidad
def detect_emotion(pitch, intensity):
    pitch_mean = np.nanmean(pitch.selected_array['frequency'])
    intensity_mean = np.nanmean(intensity.values)

    if pitch_mean > 200 and intensity_mean > 60:
        return "Feliz"
    elif pitch_mean < 150 and intensity_mean < 50:
        return "Triste"
    elif pitch_mean > 180 and intensity_mean > 70:
        return "Enojado"
    else:
        return "Neutral"

# Función para extraer información básica del archivo
def get_file_info(filepath):
    filename = os.path.basename(filepath)
    file_size = os.path.getsize(filepath) / (1024 * 1024)  # Tamaño en MB
    creation_time = datetime.fromtimestamp(os.path.getctime(filepath)).strftime('%Y-%m-%d %H:%M:%S')
    return filename, file_size, creation_time

# Función para normalizar la amplitud y frecuencia de muestreo
def normalize_audio(sound1, sound2):
    # Normalizar frecuencia de muestreo
    target_sr = max(sound1.sampling_frequency, sound2.sampling_frequency)
    sound1 = sound1.resample(target_sr)
    sound2 = sound2.resample(target_sr)

    # Recortar o rellenar para igualar la duración
    min_duration = min(sound1.get_total_duration(), sound2.get_total_duration())
    sound1 = sound1.extract_part(from_time=0, to_time=min_duration)
    sound2 = sound2.extract_part(from_time=0, to_time=min_duration)

    return sound1, sound2

# Función para comparar MFCCs usando DTW
def compare_mfcc(sound1, sound2):
    y1 = sound1.values.T.flatten()
    y2 = sound2.values.T.flatten()
    sr = sound1.sampling_frequency

    mfcc1 = librosa.feature.mfcc(y=y1, sr=sr, n_mfcc=13)
    mfcc2 = librosa.feature.mfcc(y=y2, sr=sr, n_mfcc=13)

    distance, _ = fastdtw(mfcc1.T, mfcc2.T, dist=euclidean)
    return distance

# Función principal para analizar los audios
def analyze_audio(audio1, audio2):
    try:
        # Obtener información de los archivos originales
        filename1, size1, creation1 = get_file_info(audio1)
        filename2, size2, creation2 = get_file_info(audio2)

        # Convertir los audios a WAV si no están en ese formato
        original_audio1 = audio1
        original_audio2 = audio2

        if not audio1.endswith(".wav"):
            audio1 = convert_to_wav(audio1)
        if not audio2.endswith(".wav"):
            audio2 = convert_to_wav(audio2)

        # Obtener información de los archivos convertidos
        converted_filename1, converted_size1, _ = get_file_info(audio1)
        converted_filename2, converted_size2, _ = get_file_info(audio2)

        # Cargar los audios usando PRAAT
        sound1 = parselmouth.Sound(audio1)
        sound2 = parselmouth.Sound(audio2)

        # Normalizar las señales
        sound1, sound2 = normalize_audio(sound1, sound2)

        # Extraer pitch e intensidad
        pitch1 = sound1.to_pitch()
        pitch2 = sound2.to_pitch()
        intensity1 = sound1.to_intensity()
        intensity2 = sound2.to_intensity()

        # Detectar emociones
        emotion1 = detect_emotion(pitch1, intensity1)
        emotion2 = detect_emotion(pitch2, intensity2)

        # Extraer formantes (F1, F2, F3)
        formants1 = sound1.to_formant_burg(max_number_of_formants=4)
        formants2 = sound2.to_formant_burg(max_number_of_formants=4)

        # Calcular valores promedio de pitch y formantes
        def get_pitch_and_formants(pitch, formants):
            f0_values = pitch.selected_array['frequency']
            f0_mean = np.mean(f0_values[f0_values != 0])  # Ignorar valores cero

            f1_values = [formants.get_value_at_time(1, t) for t in formants.ts()]
            f2_values = [formants.get_value_at_time(2, t) for t in formants.ts()]
            f3_values = [formants.get_value_at_time(3, t) for t in formants.ts()]

            f1_mean = np.nanmean(f1_values)
            f2_mean = np.nanmean(f2_values)
            f3_mean = np.nanmean(f3_values)

            return f0_mean, f1_mean, f2_mean, f3_mean

        f0_1, f1_1, f2_1, f3_1 = get_pitch_and_formants(pitch1, formants1)
        f0_2, f1_2, f2_2, f3_2 = get_pitch_and_formants(pitch2, formants2)

        # Comparar similitud espectral
        spectral_similarity = compare_mfcc(sound1, sound2)

        # Comparar formantes usando distancia euclidiana
        def compare_formants(f1_1, f2_1, f3_1, f1_2, f2_2, f3_2):
            distance = np.linalg.norm(np.array([f1_1, f2_1, f3_1]) - np.array([f1_2, f2_2, f3_2]))
            return distance

        formant_distance = compare_formants(f1_1, f2_1, f3_1, f1_2, f2_2, f3_2)

        # Determinar si las voces son las mismas
        mfcc_threshold = 100  # Umbral de similitud MFCC
        formant_threshold = 100  # Umbral de distancia entre formantes
        is_same_voice = (spectral_similarity < mfcc_threshold) and (formant_distance < formant_threshold)

        # Generar gráficos
        plt.figure(figsize=(12, 24))

        # Oscilograma comparativo
        plt.subplot(6, 1, 1)
        plt.plot(sound1.values[0], label="Audio 1", alpha=0.7)
        plt.plot(sound2.values[0], label="Audio 2", alpha=0.7)
        plt.title("Oscilograma Comparativo")
        plt.legend()

        # Pulsos detectados
        plt.subplot(6, 1, 2)
        plt.plot(sound1.values[0], label="Audio 1", alpha=0.7)
        plt.plot(sound2.values[0], label="Audio 2", alpha=0.7)
        plt.title("Pulsos Detectados")
        plt.legend()

        # Formantes
        plt.subplot(6, 1, 3)
        plt.plot(formants1.ts(), [formants1.get_value_at_time(1, t) for t in formants1.ts()], label="F1 Audio 1")
        plt.plot(formants1.ts(), [formants1.get_value_at_time(2, t) for t in formants1.ts()], label="F2 Audio 1")
        plt.plot(formants1.ts(), [formants1.get_value_at_time(3, t) for t in formants1.ts()], label="F3 Audio 1")
        plt.title("Formantes (Audio 1)")
        plt.legend()

        # Pitch (F0)
        plt.subplot(6, 1, 4)
        plt.plot(pitch1.selected_array['frequency'], label="F0 Audio 1")
        plt.plot(pitch2.selected_array['frequency'], label="F0 Audio 2")
        plt.title("Tono Fundamental (F0)")
        plt.legend()

        # Intensidad
        plt.subplot(6, 1, 5)
        plt.plot(intensity1.values.T, label="Intensidad Audio 1")
        plt.plot(intensity2.values.T, label="Intensidad Audio 2")
        plt.title("Intensidad")
        plt.legend()

        # Gráfico comparativo de ruido
        plt.subplot(6, 1, 6)
        plt.specgram(sound1.values[0], Fs=sound1.sampling_frequency, label="Audio 1", alpha=0.7)
        plt.specgram(sound2.values[0], Fs=sound2.sampling_frequency, label="Audio 2", alpha=0.7)
        plt.title("Comparación de Ruido")
        plt.legend()

        # Guardar gráficos
        plt.tight_layout()
        plt.savefig("comparison.png")
        print("Gráficos generados y guardados correctamente.")

        # Explicación textual del análisis
        explanation = (
            "**Archivo Original 01**\n"
            f"Nombre: {filename1}, Tamaño: {size1:.2f} MB, Fecha: {creation1}\n"
            "---------------------\n"
            "**Convertido 01**\n"
            f"Nombre: {converted_filename1}, Tamaño: {converted_size1:.2f} MB\n"
            f"Emoción: {emotion1}\n"
            "************************************************\n"
            "**Archivo Original 02**\n"
            f"Nombre: {filename2}, Tamaño: {size2:.2f} MB, Fecha: {creation2}\n"
            "---------------------\n"
            "**Convertido 02**\n"
            f"Nombre: {converted_filename2}, Tamaño: {converted_size2:.2f} MB\n"
            f"Emoción: {emotion2}\n"
            "************************************************\n"
            f"Similitud MFCC: {spectral_similarity:.2f}\n"
            f"Distancia entre formantes: {formant_distance:.2f}\n"
            "-----------------------\n"
        )
        explanation += (
            f"Conclusión: Las voces son las mismas.\n" if is_same_voice else
            f"Conclusión: Las voces no son las mismas.\n"
        )

        return explanation, "comparison.png"

    except Exception as e:
        print(f"Error durante el procesamiento: {e}")
        return f"Error: {e}", None

# Interfaz de Gradio
iface = gr.Interface(
    fn=analyze_audio,
    inputs=[gr.Audio(type="filepath"), gr.Audio(type="filepath")],
    outputs=[gr.Textbox(label="Resultado"), gr.Image(label="Gráficos")],
    title="Análisis Forense de Voz con PRAAT",
    description="""
    **Este programa está diseñado para realizar análisis forenses de voz utilizando técnicas avanzadas de fonética con la librería PRAAT.**
    Permite comparar dos muestras de audio, detectar emociones, estimar edad y sexo, y determinar si las voces pertenecen a la misma persona. Además, genera gráficos detallados para facilitar la interpretación de los resultados.

    ### Características Principales:
    - **Conversión Automática a WAV**: Convierte archivos de audio en formatos comprimidos (MP3, M4A, etc.) a WAV para un análisis preciso.
    - **Detección de Emociones**: Identifica emociones como felicidad, tristeza, enojo o neutralidad basándose en el tono (pitch) y la intensidad.
    - **Comparación de Formantes y MFCC**: Utiliza técnicas robustas como MFCC (Mel-Frequency Cepstral Coefficients) y DTW (Dynamic Time Warping) para comparar patrones de voz.
    - **Gráficos Detallados**: Incluye oscilogramas, gráficos de pitch, intensidad, formantes y espectrogramas para un análisis visual completo.
    - **Información de Archivos**: Muestra detalles de los archivos originales y convertidos para mantener la transparencia y la integridad forense.

    **Desarrollado por:**
    José R. Leonett para el **Grupo de Peritos Forenses Digitales de Guatemala**
    🌐 [www.forensedigital.gt](http://www.forensedigital.gt)
    """,
    allow_flagging="never"
)

iface.launch()

/bin/bash: line 1: nvidia-smi: command not found


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e7702274327c1da286.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
